---
title: '빅분기 실기 - 2유형 문제풀이(2)'
author: 'Hyunsoo Kim'
date: '2023-10-22'
categories: [Python, Pandas, Numpy, sklearn]
jupyter: python3
page-layout: full
---

> 실기 2 유형(2)

- 출처 : [MetaCodeM](https://www.youtube.com/watch?v=L6sSKf2sv0k)
- 참고 : [시험환경 체험](https://dataq.goorm.io/exam/116674/%EC%B2%B4%ED%97%98%ED%95%98%EA%B8%B0/quiz/4)

# 2유형 문제풀이(2)

# 제2유형_연습하기_당뇨진척정도(회귀)

## &#9989; 데이터 분석 순서
### 1. 라이브러리 및 데이터 확인
### 2. 데이터 탐색(EDA)
### 3. 데이터 전처리 및 분리
### 4. 모델링 및 성능평가
### 5. 예측값 제출

## &#9989; 1. 라이브러리 및 데이터 확인

In [41]:
import pandas as pd
import numpy as np

In [42]:
###### 실기환경 복사 영역 ######
import pandas as pd
import numpy as np
# 실기 시험 데이터셋으로 셋팅하기 (수정금지)
from sklearn.datasets import load_diabetes
# diabetes 데이터셋 로드
diabetes = load_diabetes()
x = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = pd.DataFrame(diabetes.target)

# 실시 시험 데이터셋으로 셋팅하기 (수정금지)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2,
                                                   random_state = 2023)

x_test = pd.DataFrame(x_test.reset_index())
x_train = pd.DataFrame(x_train.reset_index())
y_train = pd.DataFrame(y_train.reset_index())

x_test.rename(columns = {'index':'cust_id'}, inplace=True)
x_train.rename(columns = {'index':'cust_id'}, inplace=True)
y_train.columns = ['cust_id', 'target']
###### 실기환경 복사 영역 ######

In [43]:
# from sklearn.datasets import load_diabetes
# diabetes = load_diabetes()
# x = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
# y = pd.DataFrame(diabetes.target)

# from sklearn.model_selection import train_test_split
# x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,
#                                                 random_state=2023)

# x_test = pd.DataFrame(x_test.reset_index())
# x_train = pd.DataFrame(x_train.reset_index())
# y_train = pd.DataFrame(y_train.reset_index())

# x_test.rename(columns={'index':'cust_id'},inplace=True)
# x_train.rename(columns={'index':'cust_id'},inplace=True)
# y_train.columns=['cust']

### 🏥 당뇨병 환자의 질병 진행정도를 예측해보자
### - 데이터의 결측치, 이상치, 변수들에 대해 전처리하고
### - 회귀모델을 사용하여 Rsq, MSE 값을 산출하시오
### - 제출은 cust_id, target 변수를 가진 dataframe 형태로 제출하시오

In [44]:
# 데이터 설명
print(diabetes.DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

## &#9989; 2. 데이터 탐색(EDA)

In [45]:
# 데이터의 행/열 확인

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(353, 11)
(89, 11)
(353, 2)


In [46]:
# 초기 데이터 확인

print(x_train.head(3))
print(x_test.head(3))
print(y_train.head(3))

   cust_id       age       sex       bmi        bp        s1        s2  \
0        4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596   
1      318  0.088931 -0.044642  0.006728  0.025315  0.030078  0.008707   
2      301 -0.001882  0.050680 -0.024529  0.052858  0.027326  0.030001   

         s3        s4        s5        s6  
0  0.008142 -0.002592 -0.031991 -0.046641  
1  0.063367 -0.039493  0.009436  0.032059  
2  0.030232 -0.002592 -0.021394  0.036201  
   cust_id       age       sex       bmi        bp        s1        s2  \
0      280  0.009016  0.050680  0.018584  0.039087  0.017694  0.010586   
1      412  0.074401 -0.044642  0.085408  0.063187  0.014942  0.013091   
2       68  0.038076  0.050680 -0.029918 -0.040099 -0.033216 -0.024174   

         s3        s4        s5        s6  
0  0.019187 -0.002592  0.016305 -0.017646  
1  0.015505 -0.002592  0.006209  0.085907  
2 -0.010266 -0.002592 -0.012908  0.003064  
   cust_id  target
0        4   135.0
1      318   109

In [47]:
# 변수명과 데이터 타입이 매칭이 되는지, 결측치가 있는지 확인해보세요

print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  353 non-null    int64  
 1   age      353 non-null    float64
 2   sex      353 non-null    float64
 3   bmi      353 non-null    float64
 4   bp       353 non-null    float64
 5   s1       353 non-null    float64
 6   s2       353 non-null    float64
 7   s3       353 non-null    float64
 8   s4       353 non-null    float64
 9   s5       353 non-null    float64
 10  s6       353 non-null    float64
dtypes: float64(10), int64(1)
memory usage: 30.5 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  89 non-null     int64  
 1   age      89 non-null     float64
 2   sex      89 non-null     float64
 3   bmi      89 non-null     float64
 4   bp      

In [48]:
# x_train 과 x_test 데이터의 기초통계량을 잘 비교해보세요

print(x_train.describe())
print(x_test.describe())
print(y_train.describe())

          cust_id         age         sex         bmi          bp          s1  \
count  353.000000  353.000000  353.000000  353.000000  353.000000  353.000000   
mean   212.634561    0.000804    0.000724    0.000640   -0.000326    0.001179   
std    126.668903    0.047617    0.047673    0.048141    0.046585    0.047891   
min      0.000000   -0.107226   -0.044642   -0.084886   -0.112400   -0.126781   
25%    105.000000   -0.038207   -0.044642   -0.035307   -0.033214   -0.033216   
50%    210.000000    0.005383   -0.044642   -0.006206   -0.005671   -0.002945   
75%    322.000000    0.038076    0.050680    0.030440    0.032201    0.027326   
max    441.000000    0.110727    0.050680    0.170555    0.125158    0.153914   

               s2          s3          s4          s5          s6  
count  353.000000  353.000000  353.000000  353.000000  353.000000  
mean     0.001110   -0.000452    0.000901    0.001446    0.000589  
std      0.048248    0.048600    0.048045    0.047160    0.048122 

In [49]:
# y 데이터도 구체적으로 살펴보세요
print(y_train.head())

   cust_id  target
0        4   135.0
1      318   109.0
2      301    65.0
3      189    79.0
4      288    80.0


In [50]:
# y 데이터도 구체적으로 살펴보세요
print(y_train.describe())

          cust_id      target
count  353.000000  353.000000
mean   212.634561  152.943343
std    126.668903   75.324692
min      0.000000   37.000000
25%    105.000000   90.000000
50%    210.000000  141.000000
75%    322.000000  208.000000
max    441.000000  346.000000


## &#9989; 3. 데이터 전처리 및 분리

### 1) 결측치, 2) 이상치, 3) 변수처리하기

In [51]:
# 결측치 확인

print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

cust_id    0
age        0
sex        0
bmi        0
bp         0
s1         0
s2         0
s3         0
s4         0
s5         0
s6         0
dtype: int64
cust_id    0
age        0
sex        0
bmi        0
bp         0
s1         0
s2         0
s3         0
s4         0
s5         0
s6         0
dtype: int64
cust_id    0
target     0
dtype: int64


In [52]:
# 결측치 제거
# df = df.dropna()
# print(df)

# 참고사항
# print(df.dropna().shape) # 행 기준으로 삭제

In [53]:
# 결측치 대체 (평균값, 중앙값, 최빈값)
# 연속형 변수 :  중앙값, 평균값
# df['변수명'].median()
# df['변수명'].mean()

# 범주형 변수 :  최빈값
# df['변수명'].mode()


# df['변수명'] = df['변수명'].fillna(대체할값)

In [54]:
# 이상치 대체
# (참고) df['변수명'] = np.where(df['변수명'] >= 5, 대체할 값, df['변수명'])

In [55]:
# 변수처리

# 불필요한 변수 제거
# df = df.drop(columns = ['변수1', '변수2'])
# df = df.drop(['변수1', '변수2'], axis=1)

# 필요시 변수 추가(파생변수 생성)
# df['파생변수명'] = df['A'] * df['B'] (파생변수 생성 수식)

# 원핫인코딩(가변수 처리)
# x_train = pd.get_dummies(x_train)
# x_test = pd.get_dummies(x_test)

# print(x_train.info())
# print(x_test.info())

In [56]:
# 변수처리

# 불필요한 변수(columns) 제거
# cust_id 는 불필요한 변수이므로 제거합니다.
# 단, test셋의 cust_id가 나중에 제출이 필요하기 떄문에 별도로 저장

cust_id = x_test['cust_id'].copy()

# 각 데이터에서 cust_id 변수 제거
x_train = x_train.drop(columns = ['cust_id']) # drop(columns = ['변수1', '변수2'])
x_test = x_test.drop(columns = ['cust_id'])

###  데이터 분리

In [57]:
# 데이터를 훈련 세트와 검증용 세트로 분할 (80% 훈련, 20% 검증용)
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train['target'],
                                                   test_size = 0.2,
                                                   random_state = 23)
# 분류 모델에서는 층화(starify)를 할 필요가 없다. 연속형인 경우에만 사용

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(282, 10)
(71, 10)
(282,)
(71,)


## &#9989; 4. 모델링 및 성능평가

In [58]:
# 랜덤포레스 모델 사용 (참고 : 분류모델은 RandomForestClassifier)
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state = 2023)
model.fit(x_train, y_train)

RandomForestRegressor(random_state=2023)

In [59]:
# 모델을 사용하여 테스트 데이터 예측
y_pred = model.predict(x_val)

In [60]:
# 모델 성능 평가 (평균 제곱 오차 및 R-squared)
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_val, y_pred) # (실제값, 예측값)
r2 = r2_score(y_val, y_pred) # (실제값, 예측값)

In [61]:
# MSE(mean_squared_error, 평균 제곱 오차)
print(mse)

2571.6276845070424


In [62]:
# R2 score(R-squared)
print(r2)

0.5235611874726152


In [63]:
# RMSE (root mean squared error)
rmse = mse ** 0.5
print(rmse)

50.71121852713699


## &#9989; 5. 예측값 제출

### (주의) x_test 를 모델에 넣어 나온 예측값을 제출해야함

In [64]:
#(실기시험 안내사항)
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'cust_id':cust_id, 'target':y_result}).to_csv('0030000.csv', index=False)

# 모델을 사용하여 테스트 데이터 예측
y_result = model.predict(x_test)
result = pd.DataFrame({'cust_id':cust_id, 'target':y_result})
print(result[:5])

   cust_id  target
0      280  186.51
1      412  255.92
2       68   77.97
3      324  184.22
4      101  111.14


In [65]:
# ★tip : 데이터를 저장한 다음 불러와서 제대로 제출했는지 확인해보자
# pd.DataFrame({'result':y_result}).to_csv('수험번호.csv', index=False)
# df2 = pd.read_csv("수험번호.csv")
# print(df2.head())